The notebook `Matrix.ipynb` is included for context only.  
I was **not responsible** for its implementation. It was part of a shared academic project.  
My contribution was focused on [dataset generation/pipeline].  
Please treat this notebook as background reference, not authored code.

### Importing the dataset preprocessed on Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="{account_key}")
project = rf.workspace("t2-doo52").project("aas2")
version = project.version(1)
dataset = version.download("yolov8")



### Contour detection and matrix output

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import copy

# Function to read bounding boxes from a TXT file
def read_bounding_boxes_from_txt(file_path):
    bounding_boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            category_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            bounding_boxes.append({
                "category_id": category_id,
                "bbox": (x_center, y_center, width, height)
            })
    return bounding_boxes

# Apply the Chen algorithm for brightness normalization
def chen_algorithm(image):
    log_image = np.log1p(image.astype(np.float32))
    dct_image = cv2.dct(log_image)
    mask_size = 15
    dct_image[:mask_size, :mask_size] = 0
    idct_image = cv2.idct(dct_image)
    normalized = cv2.normalize(idct_image, None, 0, 255, cv2.NORM_MINMAX)
    return normalized.astype(np.uint8)

# Function to determine the closest element to a given point
def find_closest_element(x, y, bounding_boxes):
    min_distance = float('inf')
    closest_element = None
    for element_id, (ex, ey, ew, eh) in bounding_boxes:
        center_x, center_y = ex + ew / 2, ey + eh / 2
        distance = np.sqrt((center_x - x) ** 2 + (center_y - y) ** 2)
        if distance < min_distance:
            min_distance = distance
            closest_element = element_id
    return closest_element

# Main processing function
def process_image_and_label(image_path, label_path):
    # Read bounding boxes from the label file
    bounding_boxes = read_bounding_boxes_from_txt(label_path)

    # Load the image
    image = cv2.imread(image_path)
    image_height, image_width = image.shape[:2]

    # Convert normalized bounding boxes to pixel coordinates
    element_bounding_boxes = []
    for idx, box in enumerate(bounding_boxes):
        x_center, y_center, width, height = box["bbox"]
        x_center *= image_width
        y_center *= image_height
        width *= image_width
        height *= image_height
        x_min = int(x_center - width / 2)
        y_min = int(y_center - height / 2)
        element_bounding_boxes.append((idx, (x_min, y_min, int(width), int(height))))

    # Draw bounding boxes and their indices on the image
    image_with_boxes = copy.deepcopy(image)
    for idx, (x_min, y_min, width, height) in element_bounding_boxes:
        top_left = (x_min, y_min)
        bottom_right = (x_min + width, y_min + height)
        cv2.rectangle(image_with_boxes, top_left, bottom_right, (0, 255, 0), 2)
        cv2.putText(image_with_boxes, str(idx), (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Preprocess the image
    equalized = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(equalized)
    equalized = chen_algorithm(equalized)

    _, binary = cv2.threshold(equalized, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (10, 10))
    binary = cv2.morphologyEx(binary, cv2.MORPH_DILATE, kernel)
    edges = cv2.Canny(binary, 50, 150, apertureSize=3)

    # Find contours of the paths
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = tuple(arr for arr in contours if arr.shape[0] >= 5)

    # Draw contours on the image
    cv2.drawContours(image_with_boxes, contours, -1, (0, 255, 0), 2)

    # Initialize the connection matrix
    num_elements = len(element_bounding_boxes)
    connection_matrix = np.zeros((num_elements, num_elements), dtype=int)

    # Process each contour
    for contour in contours:
        epsilon = 0.02 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        for i in range(len(approx) - 1):
            start_point = tuple(approx[i][0])
            end_point = tuple(approx[i + 1][0])
            start_element = find_closest_element(start_point[0], start_point[1], element_bounding_boxes)
            end_element = find_closest_element(end_point[0], end_point[1], element_bounding_boxes)
            if start_element is not None and end_element is not None and start_element != end_element:
                connection_matrix[start_element][end_element] = 1
                connection_matrix[end_element][start_element] = 1  # Assuming bidirectional connections

    # Display the image with detected contours and bounding boxes
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title(f"Image: {os.path.basename(image_path)}")
    plt.show()

    # Print the connection matrix
    print(f"Connection Matrix for {os.path.basename(image_path)}:")
    print(connection_matrix)

# Folder paths
folder_path = "/content/ee-1/train"
pictures_folder = os.path.join(folder_path, "images")
labels_folder = os.path.join(folder_path, "labels")

# List all files in the pictures and labels folders
picture_files = sorted([f for f in os.listdir(pictures_folder) if f.endswith('.jpg')])
label_files = sorted([f for f in os.listdir(labels_folder) if f.endswith('.txt')])

# Ensure there is a matching label file for each picture
assert len(picture_files) == len(label_files), "Mismatch between number of pictures and labels"

# Iterate over each image-label pair
for picture_file, label_file in zip(picture_files, label_files):
    image_path = os.path.join(pictures_folder, picture_file)
    label_path = os.path.join(labels_folder, label_file)
    process_image_and_label(image_path, label_path)
